In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## Getting dataset

In [3]:
df = pd.read_csv('bank-full.csv', delimiter=';')

In [5]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [19]:
all_columns = df.columns

df.dtypes

## Features

In [25]:
# Specify the columns you want to exclude
exclude_columns = ['default', 'loan']

## Data preparation
 - Select only the features from above.
 - Check if the missing values are presented in the features.

In [26]:
# Use list comprehension to create a new list excluding the unwanted columns
selected_columns = [col for col in all_columns if col not in exclude_columns]
selected_columns

['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y']

In [33]:
selected_columns = df[selected_columns]

In [37]:
selected_columns.isnull().mean()

age          0.0
job          0.0
marital      0.0
education    0.0
balance      0.0
housing      0.0
contact      0.0
day          0.0
month        0.0
duration     0.0
campaign     0.0
pdays        0.0
previous     0.0
poutcome     0.0
y            0.0
dtype: float64

## Question 1
 - What is the most frequent observation (mode) for the column education?

In [41]:
selected_columns['education']

0         tertiary
1        secondary
2        secondary
3          unknown
4          unknown
           ...    
45206     tertiary
45207      primary
45208    secondary
45209    secondary
45210    secondary
Name: education, Length: 45211, dtype: object

# Find the mode of the 'education' column in the selected_columns DataFrame
mode_education = selected_columns['education'].mode()[0]
mode_education

## Question 2
 - Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between  every pair of features.

 - What are the two features that have the biggest correlation?

In [48]:
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [51]:
# Assuming 'selected_columns' is your DataFrame
# List of numerical features
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# Step 1: Filter the DataFrame to include only the numerical features
numerical_df = selected_columns[numerical]

# Step 2: Compute the correlation matrix
correlation_matrix = numerical_df.corr()

# Step 3: Stack the correlation matrix and find the highest correlation
correlation_values = correlation_matrix.stack().reset_index()
correlation_values.columns = ['Feature 1', 'Feature 2', 'Correlation']

# Exclude self-correlation
correlation_values = correlation_values[correlation_values['Feature 1'] != correlation_values['Feature 2']]

# Find the two features with the highest correlation
top_correlation = correlation_values.sort_values(by='Correlation', ascending=False).head(1)

# Print the result
top_correlation

,Feature 1,Feature 2,Correlation
47,previous,pdays,0.45482


## Target encoding
 - Now we want to encode the y variable.
 - Let's replace the values yes/no with 1/0.